In [5]:
# Import necessary libraries
from twelvelabs import TwelveLabs
from twelvelabs.models.task import Task
import time
import requests
import json

# Define your API key and initialize the SDK
api_key = "tlk_0SAJ5BF3G8RGXR2N1P6GZ3XPVHS2"
client = TwelveLabs(api_key=api_key)

# Set your index ID
index_id = "66b188fd2ef98f0d16b78fff"

# Base URL for the TwelveLabs API
BASE_URL = "https://api.twelvelabs.io/v1.2"

# Function to check if a video exists in the index
def check_video_exists(video_id):
    url = f"{BASE_URL}/indexes/{index_id}/videos/{video_id}"
    headers = {
        "accept": "application/json",
        "Content-Type": "application/json",
        "x-api-key": api_key
    }
    response = requests.get(url, headers=headers)
    return response.status_code == 200

# Function to upload video if it does not exist
def upload_video(video_path):
    task = client.task.create(
        index_id=index_id,
        file=video_path
    )
    task.wait_for_done(sleep_interval=5, callback=lambda t: print(f"  Status={t.status}"))
    if task.status != "ready":
        raise RuntimeError(f"Indexing failed with status {task.status}")
    print(f"The unique identifier of your video is {task.video_id}.")
    return task.video_id

# Function to summarize the video and print the result
def summarize_video(video_id):
    data = {
        "video_id": video_id,
        "type": "summary",
        "prompt": "what is shown in the beginning of the video"
    }
    response = requests.post(f"{BASE_URL}/summarize", json=data, headers={"x-api-key": api_key})
    response_data = response.json()
    print("Video Analysis Result:")
    print(response_data['summary'])

# Main workflow
video_id = "66b1917dfe45e78ff2976b22"  # This should be retrieved or known beforehand
if not check_video_exists(video_id):
    video_path = "test.mp4"
    video_id = upload_video(video_path)

summarize_video(video_id)


Video Analysis Result:
The video begins with a warning message cautioning viewers about potentially disturbing content. This message is followed by footage of a severe road accident involving multiple vehicles, including a truck and a car, on a mountainous road. The hazardous conditions, characterized by heavy rain and fog, have significantly limited visibility and contributed to the accident. The vehicles involved have sustained considerable damage. Emergency services are seen arriving at the scene, attempting to manage the situation and provide assistance to the injured. A news report overlay confirms that the accident resulted in several fatalities and numerous injuries, emphasizing the poor road conditions due to the weather. The video continues to show the aftermath, with emergency responders working to clear the scene. The video concludes with a final warning message, reminding viewers of the graphic nature of the content they have just witnessed.
